# Project Part 1

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)



## Introduction/Background

In this project I will attempting to train a model to predict news media bias. In order to do this I will be utilizing the dataset NLPCSS-20 from the Association for Computational Linguistics which can be found on Github. The dataset is a collection or corpus contrived of 6,964 news articles. The labels of how we describe the corpus are the following:
- title: title of news article
- content: content of news article
- source: source of the content
- allsides-bias: bias (left, center or right)
- misc: aurhor, date, etc
- adfontes_fair: how fair is the news article (bias, neutral or unknown)
- adfontes_political: how political bias skewed the article is (bias, neutral or unknown)
- event_id:event id

This is a text classification problem as how we are trying to classify combination of strings that demonstrate wheter a news content is politically bias or not. The combination of strings is the content itself, it is then labeled with how bias that content is. 


## Exploratory Data Analysis

First step is to analyze the data we have in order to figure out how our data looks like and what it is trying to tell us.

The dataset comes in a Json file so there are extra steps as to converting the data into a dataframe.

In [14]:
# import all of the python modules/packages you'll need here
import nltk
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

#nltk.download('wordnet')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/msalvador45/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/msalvador45/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/msalvador45/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

As we can oberseve the json data set is not able to be formatted into a dataframe, this is becuase its format is unsupported by pandas and will have to be reformated.

In [ ]:
# convert json file to dataframe
# path has original data, path2 is a small excerpt from og data w/ a fix to check if it is able to reformatt into df
path= "NLPCSS-20-main/data/released_data.json"
path2= "NLPCSS-20-main/data/example.json"

# test_df = pd.read_json(path, orient='records')
test_df2 = pd.read_json(path2, orient='records')

# print(test_df)
print (test_df2)

In [ ]:
#path to reformatted json file
path_data= 'NLPCSS-20-main/data/data.json'
#reformatt json file
with open(path, 'r') as rf:
    with open(path_data, 'w') as wf:
        #add [ at the beginning of the corpus
        wf.write('[\n')
        #add a comma at the end of each line
        lines= rf.read().splitlines()
        new_line= ',\n'.join(lines)
        wf.write(new_line)
        #add ] at the end of the corpus
        wf.write('\n]')
        


We can now convert data into a pandas dataframe and checkout initial observations

In [ ]:
"""Convert reformatted dataset into dataframe"""
df = pd.read_json(path_data, orient='records')

df.head(3)

Now lets do some basic pandas data exploration to get a sense of the data.
There are 7775 observations with 8 columns, but it seems as we might not have to use all columns.

In [ ]:
print(df.info())
print("\nThe shape of dataset as of right now is ", df.shape)


We want to deal with a simple model first so we will proceed to only use 'content' and 'allsides_bias' first, and we can drop every other column.

In [ ]:
df.drop(columns=['source','title','event_id','misc','adfontes_fair','adfontes_political'], inplace=True)
#lowercase the content column
df['content']= df['content'].str.lower()
df.head()

Lets see the shape and columns of our remodified data frame as well as the unique values of our allsides_bias and its value counts.

In [ ]:
print(df.info())
print("\nThe shape of dataset as of right now is ", df.shape,"\n")
print("The unique values for allsides_bias is:\n",df.allsides_bias.unique(),"\n")
df.allsides_bias.value_counts(normalize=True)

We can see most results come from the left, around 37% com from the right and 15% are center biased.

Now lets check for empty values in both columns.

In [ ]:
null_entries= df.isnull().sum()
null_entries

So our data frame as of now has two rows, and both contain values that are not empty or NaN.

Lets take a look into the stopwords vs non-stopwords within the dataset

In [ ]:
"Create a corpus without stopwords"
from nltk.corpus import stopwords
stops= set(stopwords.words('english'))

corpus=[]
new= df['content'].str.split()
new= new.values.tolist()
corpus=[word for i in new for word in i]

from collections import defaultdict
dic= defaultdict(int)
for word in corpus:
    if word in stops:
        dic[word]+=1

top= sorted(dic.items(), key=lambda x:x[1], reverse= True)[:10]
x,y= zip(*top)
print("Bar graph of amt of stopwords in content:\n")
plt.bar(x,y)

In [ ]:
"create a chart to show number of occurences of words in dataset"
from collections import Counter

counter= Counter(corpus)
most= counter.most_common()

x,y= [], []
for word, count in most[:40]:
    if (word not in stops):
        x.append(word)
        y.append(count)

sns.barplot(x=y, y=x)

# Project Part II

Before making a baseline for the model, we will want to process the dataset we will have to tokenize our data.

In [ ]:
"Using nltk tokenize the dataset"
from nltk.tokenize import word_tokenize
nltk.download('punkt')

df['tokens_raw']= df['content'].apply(word_tokenize)
df.head()

As well as removing stopwords from the dataset

In [ ]:
import re

chars2remove= set(['.','!','/','?','#','-']) 
#stops contains our stopwords
df['tokens_raw']= df['tokens_raw'].apply(lambda x: [w for w in x if w not in stops])
df['tokens_raw']= df['tokens_raw'].apply(lambda x: [w for w in x if w not in chars2remove])
df['tokens_raw']= df['tokens_raw'].apply(lambda x: [w for w in x if not re.match('^http',w)])
df['tokens_raw']= df['tokens_raw'].apply(lambda x: [w for w in x if not re.match('^@', w)])
df.head()

Now we will proceed to lemmetize our tokens

In [16]:
from nltk.stem import WordNetLemmatizer

lemmatizer= WordNetLemmatizer()
df['tokens']= df['tokens_raw'].apply(lambda x: [lemmatizer.lemmatize(w, pos="v") for w in x])
df.head()


TypeError: lemmatize() got an unexpected keyword argument 'ps'